In [12]:
import numpy as np
import pandas as pd
import ccxt
import time
import dateutil

# define the market
exchange_f = ccxt.binance({
    'proxies': {
        'http': 'http://localhost:1080',
        'https': 'http://localhost:1080',
    },
})

symbol = 'USDS/USDT'
timeframe = "1h"
limit = 1000
df = pd.DataFrame(exchange_f.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit))
df['symbol'] = symbol
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Symbol']
print(df)

df['Datetime'] = df['Datetime'].apply(
    lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))

df['ema_12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['ema_144'] = df['Close'].ewm(span=144, adjust=False).mean()
df['ema_169'] = df['Close'].ewm(span=169, adjust=False).mean()
df['ema_576'] = df['Close'].ewm(span=576, adjust=False).mean()
df['ema_676'] = df['Close'].ewm(span=676, adjust=False).mean()

df['ema_12_lag'] = df['ema_12'].shift(1)
df['ema_144_lag'] = df['ema_144'].shift(1)

# Check if the conditions for a bullish trend are met
is_bullish_trend = (
    (df['ema_12_lag'] < df['ema_144_lag'])
    & (
        (df['Close'] > df['ema_12'])
        & (df['ema_12'] > df['ema_144'])
        & (df['ema_144'] > df['ema_169'])
        & (df['ema_169'] > df['ema_576'])
        & (df['ema_576'] > df['ema_676'])
    )
)

# Update the 'is_bullish' column
df.loc[is_bullish_trend, 'is_bullish'] = True

df[df['is_bullish']==True]

          Datetime    Open    High     Low   Close        Vol      Symbol
0    1675702800000  1.9691  2.0128  1.9587  2.0128   834416.6  MAGIC/USDT
1    1675706400000  2.0125  2.0125  1.9824  1.9940   492441.9  MAGIC/USDT
2    1675710000000  1.9930  1.9974  1.9546  1.9682   605412.9  MAGIC/USDT
3    1675713600000  1.9682  1.9909  1.9558  1.9898   374935.0  MAGIC/USDT
4    1675717200000  1.9895  1.9910  1.9237  1.9434   733823.7  MAGIC/USDT
..             ...     ...     ...     ...     ...        ...         ...
995  1679284800000  1.7280  1.7623  1.6976  1.7563  1816104.9  MAGIC/USDT
996  1679288400000  1.7563  1.7727  1.7530  1.7603  1025436.7  MAGIC/USDT
997  1679292000000  1.7604  1.7716  1.7344  1.7613  1562884.0  MAGIC/USDT
998  1679295600000  1.7614  1.8226  1.7589  1.8149  1735354.7  MAGIC/USDT
999  1679299200000  1.8149  1.8297  1.7751  1.7821  1099413.2  MAGIC/USDT

[1000 rows x 7 columns]


,Datetime,Open,High,Low,Close,Vol,Symbol,ema_12,ema_144,ema_169,ema_576,ema_676,ema_12_lag,ema_144_lag,is_bullish
